################################################################################################################
  
  FINAL script used to convert OHSUMED test collection to JSON format
  
################################################################################################################


In [1]:
import os
import sys
import json 

#sys.path.append("./hugging/lib/python3.7/site-packages")

In [2]:
import pandas as pd
import numpy as np

In [17]:
trainpath = "ohsumed.88-91"

#trainpath = "ohsumed.87"
#trainpath = "ohsumed.88-91"

testpath = "sample.87"

In [18]:
train_id = list()
with open(trainpath) as f:
    for line in f.readlines():
        line = line.strip()
        if line.startswith(".I"):
            line = line.replace(".I", "").strip()
            train_id.append(line)
print(f"Number of lines in train: {len(train_id)}")

test_id = list()
with open(testpath) as f:
    for line in f.readlines():
        line = line.strip()
        if line.startswith(".I"):
            line = line.replace(".I", "").strip()
            test_id.append(line)
print(f"Number of lines in test: {len(test_id)}")

Number of lines in train: 293856
Number of lines in test: 15


In [19]:
#   87 - 54710 

#   88-91 - 2,93,856
train_id

['54711',
 '54712',
 '54713',
 '54714',
 '54715',
 '54716',
 '54717',
 '54718',
 '54719',
 '54720',
 '54721',
 '54722',
 '54723',
 '54724',
 '54725',
 '54726',
 '54727',
 '54728',
 '54729',
 '54730',
 '54731',
 '54732',
 '54733',
 '54734',
 '54735',
 '54736',
 '54737',
 '54738',
 '54739',
 '54740',
 '54741',
 '54742',
 '54743',
 '54744',
 '54745',
 '54746',
 '54747',
 '54748',
 '54749',
 '54750',
 '54751',
 '54752',
 '54753',
 '54754',
 '54755',
 '54756',
 '54757',
 '54758',
 '54759',
 '54760',
 '54761',
 '54762',
 '54763',
 '54764',
 '54765',
 '54766',
 '54767',
 '54768',
 '54769',
 '54770',
 '54771',
 '54772',
 '54773',
 '54774',
 '54775',
 '54776',
 '54777',
 '54778',
 '54779',
 '54780',
 '54781',
 '54782',
 '54783',
 '54784',
 '54785',
 '54786',
 '54787',
 '54788',
 '54789',
 '54790',
 '54791',
 '54792',
 '54793',
 '54794',
 '54795',
 '54796',
 '54797',
 '54798',
 '54799',
 '54800',
 '54801',
 '54802',
 '54803',
 '54804',
 '54805',
 '54806',
 '54807',
 '54808',
 '54809',
 '54810',


In [20]:
counter =0
with open(testpath) as f:
    for line in f.readlines():
        line = line.strip()
        print(line)
        if line.startswith(".I"):
            counter+=1
            
            if counter > 20:
                break
            
print(f"Number of lines in test: {len(test_id)}")

.I 1
.U
87049087
.S
Am J Emerg Med 8703; 4(6):491-5
.M
Allied Health Personnel/*; Electric Countershock/*; Emergencies; Emergency Medical Technicians/*; Human; Prognosis; Recurrence; Support, U.S. Gov't, P.H.S.; Time Factors; Transportation of Patients; Ventricular Fibrillation/*TH.
.T
Refibrillation managed by EMT-Ds: incidence and outcome without paramedic back-up.
.P
JOURNAL ARTICLE.
.W
Some patients converted from ventricular fibrillation to organized rhythms by defibrillation-trained ambulance technicians (EMT-Ds) will refibrillate before hospital arrival. The authors analyzed 271 cases of ventricular fibrillation managed by EMT-Ds working without paramedic back-up. Of 111 patients initially converted to organized rhythms, 19 (17%) refibrillated, 11 (58%) of whom were reconverted to perfusing rhythms, including nine of 11 (82%) who had spontaneous pulses prior to refibrillation. Among patients initially converted to organized rhythms, hospital admission rates were lower for patien

 Run a loop through each of the train & test files & extract json

 The original field definiations

| Column market | Defination | key name | Notes | 

|----|----|----|----|----|

| .I | sequential identifier | seqId | important note: documents should be processed in this order |

| .U | MEDLINE identifier (UI) | medline_ui | (<DOCNO> used for relevance judgements) |
    
| .M | Human-assigned MeSH terms (MH) | mesh_terms
    
| .T | Title (TI) | title
    
| .P | Publication type (PT) | publication_type
    
| .W | Abstract (AB) | abstract
    
| .A | Author (AU) | author
    
| .S |Source (SO) | source

In [23]:
column_map = {
    ".I": "seq_id", ".U": "medline_ui", ".M": "mesh_terms", ".T": "title", 
    ".P": "publication_type", ".W": "abstract", ".A": "author",
    ".S": "source"
}

In [24]:
def ohsumed_dict():
    # Define a dict
    ohs_dict = {"seq_id": -1,
    "medline_ui": -1,
    "mesh_terms": list(),
    "title": "",
    "publication_type": list(),
    "abstract": "",
    "author": list(),
    "source": ""}
    
    return ohs_dict

In [25]:
%%time

print_tag = False

for filepath in [trainpath]:
    abstract = list()
    tag = ""

    with open(filepath) as f:
        for line in f.readlines():

            line = line.strip()

            if tag and tag != ".I" and not line.startswith("."):
                key = column_map[tag]
                if tag == ".U":
                    ohs_dict[key] = int(line)
                elif tag == ".M" or tag == ".A" or tag == ".P":
                    ohs_dict[key] = [l.strip() for l in line.split(";")]
                else:
                    ohs_dict[key] = line
                if print_tag == True:
                    print(key, line)

            if line.startswith(".I"):
                tag = ".I"
                try:
                    if ohs_dict:
                        abstract.append(ohs_dict)
                except:
                    print("first run")
                    

                ohs_dict = ohsumed_dict()
                line = line.replace(".I", "").strip()
                ohs_dict['seq_id'] = int(line)

            elif ".U" in line:
                tag = ".U"
            elif ".M" in line:
                tag = ".M"
            elif ".T" in line:
                tag = ".T"
            elif ".P" in line:
                tag = ".P"
            elif ".W" in line:
                tag = ".W"
            elif ".A" in line:
                tag = ".A"
            elif ".S" in line:
                tag = ".S"

        # Collect the last abstract
        abstract.append(ohs_dict) 
        print(f"Length of {filepath}: {len(abstract)}" )
            
        if 'sample' in filepath:
            filename = "sample.json"
        elif "87" in filepath:
            filename = "ohsumed-87.json"
        else:
            filename = "ohsumed-88-91.json"
      
        
        result = pd.DataFrame(abstract).to_json(orient="records")
        
        parsed = json.loads(result)
        with open(filename, 'w') as outfile:
            print(f"Writing to file {filename}...")
            json.dump(parsed, outfile, indent=4)
        ohs_dict = ""

Length of ohsumed.88-91: 293856
Writing to file ohsumed-88-91.json...
Wall time: 1min 9s


In [30]:
#   87 - 54710 

#   88-91 - 2,93,856
abstract[200000]

{'seq_id': 254711,
 'medline_ui': 90144721,
 'mesh_terms': ['Administration, Oral',
  'Adult',
  'Antibiotics/*TU',
  'Antibiotics, Combined/TU',
  'Bacteria/IP',
  'Cephalothin/AD/TU',
  'Comparative Study',
  'Female',
  'Human',
  'Injections, Intraperitoneal',
  'Male',
  'Middle Age',
  'Ofloxacin/AD/TU',
  'Peritoneal Dialysis, Continuous Ambulatory/*AE',
  'Peritonitis/*DT/ET/MI',
  'Prospective Studies',
  'Randomized Controlled Trials',
  'Rifampin/AD/TU',
  "Support, Non-U.S. Gov't",
  'Tobramycin/AD/TU.'],
 'title': 'A randomized prospective trial of three different regimens of treatment of peritonitis in patients on continuous ambulatory peritoneal dialysis.',
 'publication_type': ['CLINICAL TRIAL',
  'JOURNAL ARTICLE',
  'RANDOMIZED CONTROLLED TRIAL.'],
 'abstract': 'A randomized prospective study was undertaken in patients on continuous ambulatory peritoneal dialysis (CAPD) to evaluate the efficacy of three different antibiotic regimens for the treatment of peritonitis. T